# 1. load required packages

The link to get [ImageAnalysis3](https://github.com/zhengpuas47/ImageAnalysis3) 

or the Zhuang lab archived [source_tools](https://github.com/ZhuangLab/Chromatin_Analysis_2020_cell/tree/master/sequential_tracing/source)

In [5]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

import pandas as pd

# library design specific tools
from ImageAnalysis3.library_tools import LibraryDesigner as ld
from ImageAnalysis3.library_tools import LibraryTools as lt
# biopython imports
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

103308


# 2. Load transcriptome

In [6]:
## Define the genome reference path
# mouse GRCm38 genome
reference_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl'
transcriptome_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Transcriptome'
# Library directories
pool_folder = r'L:\Shiwei\DNA_MERFISH_analysis\SMARTer_nuclei_MOp'

cdna_fasta_file = os.path.join(transcriptome_folder,'Mus_musculus.GRCm38.cdna.all.fa')
ncrna_fasta_file = os.path.join(transcriptome_folder,'Mus_musculus.GRCm38.ncrna.fa')

In [7]:
def load_transcriptome_fasta_into_dataframe(fasta_file):
    d = {'transcript_id':[], 'seq_type':[], 'location':[], 'gene_id':[],
        'gene_biotype':[], 'transcript_biotype':[], 'gene_symbol':[],
        'description':[], 'seq_length':[], 'sequence':[]}
    
    try:
        for record in SeqIO.parse(fasta_file, 'fasta'):
            split_rd = record.description.split()            
            d['transcript_id'].append(split_rd[0].split('.')[0])
            d['seq_type'].append(split_rd[1])
            d['location'].append(split_rd[2])
            d['gene_id'].append(split_rd[3][5:].split('.')[0])
            d['gene_biotype'].append(split_rd[4][13:])
            d['transcript_biotype'].append(split_rd[5][19:])
            if len(split_rd) > 6:
                d['gene_symbol'].append(split_rd[6][12:])
            else:
                d['gene_symbol'].append(pd.NA)
            if len(split_rd) > 7:
                d['description'].append(' '.join(split_rd[7:])[12:])
            else:
                d['description'].append(pd.NA)
            
            d['seq_length'].append(len(str(record.seq)))
            d['sequence'].append(str(record.seq))
    
    except:
        print(split_rd)
        raise
    
    df = pd.DataFrame.from_dict(d)
    df = df.set_index('transcript_id')
    return df

In [8]:
# Load the transcriptome fasta
transcriptome_cdna = load_transcriptome_fasta_into_dataframe(cdna_fasta_file)
transcriptome_ncrna = load_transcriptome_fasta_into_dataframe(ncrna_fasta_file)
transcriptome = pd.concat((transcriptome_cdna, transcriptome_ncrna), axis=0) 

In [9]:
print(len(transcriptome))
transcriptome.head()

141450


,seq_type,location,gene_id,gene_biotype,transcript_biotype,gene_symbol,description,seq_length,sequence
transcript_id,,,,,,,,,
ENSMUST00000177564,cdna,chromosome:GRCm38:14:54122226:54122241:1,ENSMUSG00000096176,TR_D_gene,TR_D_gene,Trdd2,T cell receptor delta diversity 2 [Source:MGI ...,16,ATCGGAGGGATACGAG
ENSMUST00000196221,cdna,chromosome:GRCm38:14:54113468:54113476:1,ENSMUSG00000096749,TR_D_gene,TR_D_gene,Trdd1,T cell receptor delta diversity 1 [Source:MGI ...,9,ATGGCATAT
ENSMUST00000179664,cdna,chromosome:GRCm38:14:54113468:54113478:1,ENSMUSG00000096749,TR_D_gene,processed_transcript,Trdd1,T cell receptor delta diversity 1 [Source:MGI ...,11,ATGGCATATCA
ENSMUST00000178537,cdna,chromosome:GRCm38:6:41533201:41533212:1,ENSMUSG00000095668,TR_D_gene,TR_D_gene,Trbd1,"T cell receptor beta, D region 1 [Source:MGI S...",12,GGGACAGGGGGC
ENSMUST00000178862,cdna,chromosome:GRCm38:6:41542163:41542176:1,ENSMUSG00000094569,TR_D_gene,TR_D_gene,Trbd2,"T cell receptor beta, D region 2 [Source:MGI S...",14,GGGACTGGGGGGGC


# 3. Process MOp genes (genes obtained from smart_seq)

In [10]:
import scanpy as sc
scRNA_folder =r'L:\Shiwei\DNA_MERFISH_analysis\SMARTer_nuclei_MOp'
# load from here for saved h5ad
adata = sc.read(os.path.join(scRNA_folder,r'MOp_smart_sn_raw.h5ad'))

Mop_genes = adata.var.index.tolist()
len(Mop_genes)

33262

In [29]:
# load gene from other notebook
# retrieve genes from the MOp_smart_sn_labeled.h5ad 

#%store -r Mop_genes

In [11]:
len(Mop_genes)

33262

In [12]:
_strands = [1,1,1,1,-1]
import statistics
from statistics import mode
mode(_strands)

1

In [13]:
from tqdm.notebook import tqdm

In [15]:
import statistics
from statistics import mode


# find corresponding gene info from the reference dict
Mop_gene_dict ={}
Mop_gene_dict['gene']=[]
Mop_gene_dict['chr']=[]
Mop_gene_dict['start']=[]
Mop_gene_dict['end']=[]
Mop_gene_dict['gene_biotype']=[]
Mop_gene_dict['coding_strand']=[]

ref_genes = np.unique(transcriptome['gene_symbol'].tolist())


not_found=0
for _gene in tqdm(Mop_genes[:]):
    Mop_gene_dict['gene'].append(_gene)
    
    if _gene in ref_genes:
        _gene_ref_df = transcriptome[transcriptome['gene_symbol']==_gene]
        _location_list = _gene_ref_df['location'].tolist()
        # check format
        if 'chromosome:GRCm38:' == _location_list[0][:18]:
            _start_list = [int(_loc.split('chromosome:GRCm38:')[-1].split(':')[1]) for _loc in _location_list]
            _end_list = [int(_loc.split('chromosome:GRCm38:')[-1].split(':')[2]) for _loc in _location_list]
            Mop_gene_dict['start'].append(np.min(_start_list))
            Mop_gene_dict['end'].append(np.max(_end_list))
            Mop_gene_dict['chr'].append(_location_list[0].split('chromosome:GRCm38:')[-1].split(':')[0])
            # add transcript strand info; use most commen strand
            _strands = [int(_loc.split(':')[-1]) for _loc in _location_list]
            Mop_gene_dict['coding_strand'].append(mode(_strands))
            
        else:
            print (f'Wrong format for {_gene}. Check the result.')
            Mop_gene_dict['start'].append(pd.NA)
            Mop_gene_dict['end'].append(pd.NA)
            Mop_gene_dict['chr'].append(pd.NA)
            Mop_gene_dict['coding_strand'].append(pd.NA)
            
            
        _biotype_list = np.unique(_gene_ref_df['gene_biotype'].tolist())
        if len(_biotype_list)==1:
            Mop_gene_dict['gene_biotype'].append(_biotype_list[0])
        else:
            print(f'Multiple gene biotype exists for {_gene}. Merge the result.')
            merged_biotype = ';'.join(_biotype_list)
            Mop_gene_dict['gene_biotype'].append(merged_biotype)
            
    else:
        print(f'The gene {_gene} not found in the database. Append empty values.')
        Mop_gene_dict['start'].append(pd.NA)
        Mop_gene_dict['end'].append(pd.NA)
        Mop_gene_dict['chr'].append(pd.NA)
        Mop_gene_dict['gene_biotype'].append(pd.NA)
        Mop_gene_dict['coding_strand'].append(pd.NA)
        
        not_found+=1

print (f'There are {not_found} genes not found and were skipped from this query.')            

The gene 0610007P14Rik not found in the database. Append empty values.
The gene 0610009O20Rik not found in the database. Append empty values.
The gene 0610010B08Rik not found in the database. Append empty values.
The gene 0610011F06Rik not found in the database. Append empty values.
The gene 0610037L13Rik not found in the database. Append empty values.
The gene 0610039H22Rik not found in the database. Append empty values.
The gene 1010001N08Rik not found in the database. Append empty values.
The gene 1110001J03Rik not found in the database. Append empty values.
The gene 1110004E09Rik not found in the database. Append empty values.
The gene 1110007C09Rik not found in the database. Append empty values.
The gene 1110008F13Rik not found in the database. Append empty values.
The gene 1110008L16Rik not found in the database. Append empty values.
The gene 1110034G24Rik not found in the database. Append empty values.
The gene 1110037F02Rik not found in the database. Append empty values.
The ge

The gene 2310003H01Rik not found in the database. Append empty values.
The gene 2310009A05Rik not found in the database. Append empty values.
The gene 2310014L17Rik not found in the database. Append empty values.
The gene 2310034P14Rik not found in the database. Append empty values.
The gene 2310035C23Rik not found in the database. Append empty values.
The gene 2310036O22Rik not found in the database. Append empty values.
The gene 2310045N01Rik not found in the database. Append empty values.
The gene 2310047M10Rik not found in the database. Append empty values.
The gene 2310061J03Rik not found in the database. Append empty values.
The gene 2310067B10Rik not found in the database. Append empty values.
The gene 2310067E19Rik not found in the database. Append empty values.
The gene 2310069G16Rik not found in the database. Append empty values.
The gene 2310075K07Rik not found in the database. Append empty values.
The gene 2410004N09Rik not found in the database. Append empty values.
The ge

The gene 4930543D07Rik not found in the database. Append empty values.
The gene 4930552P12Rik not found in the database. Append empty values.
The gene 4930553C11Rik not found in the database. Append empty values.
The gene 4930557B15Rik not found in the database. Append empty values.
The gene 4930563D23Rik not found in the database. Append empty values.
Multiple gene biotype exists for 4930563F08Rik. Merge the result.
The gene 4930570G05Rik not found in the database. Append empty values.
The gene 4930572J10Rik not found in the database. Append empty values.
The gene 4930573O16Rik not found in the database. Append empty values.
The gene 4930577N17Rik not found in the database. Append empty values.
The gene 4930578C19Rik not found in the database. Append empty values.
The gene 4930594C11Rik not found in the database. Append empty values.
Multiple gene biotype exists for 4930594M22Rik. Merge the result.
The gene 4931402G19Rik not found in the database. Append empty values.
The gene 4931403

The gene A630075F10Rik not found in the database. Append empty values.
The gene A630077J23Rik not found in the database. Append empty values.
The gene A630081J09Rik not found in the database. Append empty values.
Multiple gene biotype exists for A730006G06Rik. Merge the result.
The gene A730013G03Rik not found in the database. Append empty values.
The gene A730017C20Rik not found in the database. Append empty values.
The gene A730067D02Rik not found in the database. Append empty values.
The gene A730089K16Rik not found in the database. Append empty values.
The gene A730090H04Rik not found in the database. Append empty values.
The gene A730098P11Rik not found in the database. Append empty values.
The gene A830010M20Rik not found in the database. Append empty values.
The gene A830021M18Rik not found in the database. Append empty values.
The gene A830080D01Rik not found in the database. Append empty values.
The gene A930011G23Rik not found in the database. Append empty values.
The gene A9

The gene Bmp4-ps not found in the database. Append empty values.
The gene Bre not found in the database. Append empty values.
The gene Btg1-ps2 not found in the database. Append empty values.
The gene Bzrap1 not found in the database. Append empty values.
The gene C030016D13Rik not found in the database. Append empty values.
The gene C030023E24Rik not found in the database. Append empty values.
The gene C030039L03Rik not found in the database. Append empty values.
The gene C130023O10Rik not found in the database. Append empty values.
The gene C130030K03Rik not found in the database. Append empty values.
The gene C130060K24Rik not found in the database. Append empty values.
The gene C230004F18Rik not found in the database. Append empty values.
The gene C230029M16 not found in the database. Append empty values.
The gene C230052I12Rik not found in the database. Append empty values.
The gene C230091D08Rik not found in the database. Append empty values.
The gene C330006A16Rik not found in t

...

The gene LOC105246375 not found in the database. Append empty values.
The gene LOC105246377 not found in the database. Append empty values.
The gene LOC105246380 not found in the database. Append empty values.
The gene LOC105246383 not found in the database. Append empty values.
The gene LOC105246385 not found in the database. Append empty values.
The gene LOC105246386 not found in the database. Append empty values.
The gene LOC105246388 not found in the database. Append empty values.
The gene LOC105246389 not found in the database. Append empty values.
The gene LOC105246390 not found in the database. Append empty values.
The gene LOC105246393 not found in the database. Append empty values.
The gene LOC105246394 not found in the database. Append empty values.
The gene LOC105246395 not found in the database. Append empty values.
The gene LOC105246396 not found in the database. Append empty values.
The gene LOC105246397 not found in the database. Append empty values.
The gene LOC10524639

The gene LOC105246831 not found in the database. Append empty values.
The gene LOC105246832 not found in the database. Append empty values.
The gene LOC105246835 not found in the database. Append empty values.
The gene LOC105246836 not found in the database. Append empty values.
The gene LOC105246839 not found in the database. Append empty values.
The gene LOC105246840 not found in the database. Append empty values.
The gene LOC105246841 not found in the database. Append empty values.
The gene LOC105246843 not found in the database. Append empty values.
The gene LOC105246844 not found in the database. Append empty values.
The gene LOC105246845 not found in the database. Append empty values.
The gene LOC105246846 not found in the database. Append empty values.
The gene LOC105246847 not found in the database. Append empty values.
The gene LOC105246848 not found in the database. Append empty values.
The gene LOC105246849 not found in the database. Append empty values.
The gene LOC10524685

The gene LOC432842 not found in the database. Append empty values.
The gene LOC433198 not found in the database. Append empty values.
The gene LOC436100 not found in the database. Append empty values.
The gene LOC545466 not found in the database. Append empty values.
The gene LOC545966 not found in the database. Append empty values.
The gene LOC547334 not found in the database. Append empty values.
The gene LOC628147 not found in the database. Append empty values.
The gene LOC73899 not found in the database. Append empty values.
The gene Lace1 not found in the database. Append empty values.
The gene Large not found in the database. Append empty values.
The gene Ldoc1l not found in the database. Append empty values.
The gene Lect1 not found in the database. Append empty values.
The gene Lincenc1 not found in the database. Append empty values.
The gene Lincpint not found in the database. Append empty values.
The gene Lincred1 not found in the database. Append empty values.
The gene Lincr

The gene Pcnxl2 not found in the database. Append empty values.
The gene Pcnxl3 not found in the database. Append empty values.
The gene Pcnxl4 not found in the database. Append empty values.
The gene Pddc1 not found in the database. Append empty values.
The gene Peo1 not found in the database. Append empty values.
The gene Pet2 not found in the database. Append empty values.
The gene Phf5b-ps not found in the database. Append empty values.
The gene Phxr4 not found in the database. Append empty values.
The gene Pla2g16 not found in the database. Append empty values.
The gene Pomc-ps1 not found in the database. Append empty values.
The gene Ppap2a not found in the database. Append empty values.
The gene Ppap2b not found in the database. Append empty values.
The gene Ppap2c not found in the database. Append empty values.
The gene Ppapdc1a not found in the database. Append empty values.
The gene Ppapdc1b not found in the database. Append empty values.
The gene Ppapdc2 not found in the dat

The gene Taf4a not found in the database. Append empty values.
The gene Tbrg3 not found in the database. Append empty values.
The gene Tceb1 not found in the database. Append empty values.
The gene Tceb2 not found in the database. Append empty values.
The gene Tceb3 not found in the database. Append empty values.
The gene Tcra not found in the database. Append empty values.
The gene Tcrb not found in the database. Append empty values.
The gene Tcrg not found in the database. Append empty values.
The gene Tdpx-ps1 not found in the database. Append empty values.
The gene Tenc1 not found in the database. Append empty values.
Multiple gene biotype exists for Terc. Merge the result.
The gene Tex13 not found in the database. Append empty values.
The gene Tex40 not found in the database. Append empty values.
The gene Tfdp1-ps not found in the database. Append empty values.
The gene Tldc1 not found in the database. Append empty values.
The gene Tmem110 not found in the database. Append empty v

In [16]:
Mop_gene_df = pd.DataFrame.from_dict(Mop_gene_dict)
Mop_gene_df = Mop_gene_df.set_index('gene')

Mop_gene_df

,chr,start,end,gene_biotype,coding_strand
gene,,,,,
0610005C13Rik,7,45567794,45575327,antisense,-1
0610007P14Rik,<NA>,<NA>,<NA>,<NA>,<NA>
0610009B22Rik,11,51685386,51688874,protein_coding,-1
0610009E02Rik,2,26445696,26459390,processed_transcript,1
0610009L18Rik,11,120348678,120351190,bidirectional_promoter_lncRNA,1
...,...,...,...,...,...
Zyx,6,42349630,42360213,protein_coding,1
Zzef1,11,72796226,72927120,protein_coding,1
Zzz3,3,152395473,152462826,protein_coding,1


In [18]:
Mop_gene_df.dropna()

,chr,start,end,gene_biotype,coding_strand
gene,,,,,
0610005C13Rik,7,45567794,45575327,antisense,-1
0610009B22Rik,11,51685386,51688874,protein_coding,-1
0610009E02Rik,2,26445696,26459390,processed_transcript,1
0610009L18Rik,11,120348678,120351190,bidirectional_promoter_lncRNA,1
0610010F05Rik,11,23564961,23633639,protein_coding,-1
...,...,...,...,...,...
Zyg11b,4,108229724,108301096,protein_coding,-1
Zyx,6,42349630,42360213,protein_coding,1
Zzef1,11,72796226,72927120,protein_coding,1


# 4. Add formated column for downstream analysis

In [19]:
# load gene annotation (covering all genes from the SMART-seq) for chr locus
#pool_folder = r'L:\Shiwei\DNA_MERFISH_analysis\SMARTer_nuclei_MOp'
#clean_gene_df = pd.read_csv(os.path.join(pool_folder, "MOp_smart_sn_gene_chr_info_NEW_from_transcriptome.csv"), index_col=0)

clean_gene_df = Mop_gene_df

# Append a column for gene length
clean_gene_df['length']=clean_gene_df['end']-clean_gene_df['start']
clean_gene_df=clean_gene_df.dropna()

clean_gene_df['start'] = clean_gene_df['start'].map(lambda x: int(x))
clean_gene_df['end'] = clean_gene_df['end'].map(lambda x: int(x))
clean_gene_df['length'] = clean_gene_df['length'].map(lambda x: int(x))

print(len(clean_gene_df))
clean_gene_df.head()

24596


<ipython-input-19-0009adc8161e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_gene_df['start'] = clean_gene_df['start'].map(lambda x: int(x))
<ipython-input-19-0009adc8161e>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_gene_df['end'] = clean_gene_df['end'].map(lambda x: int(x))
<ipython-input-19-0009adc8161e>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

,chr,start,end,gene_biotype,coding_strand,length
gene,,,,,,
0610005C13Rik,7,45567794,45575327,antisense,-1,7533
0610009B22Rik,11,51685386,51688874,protein_coding,-1,3488
0610009E02Rik,2,26445696,26459390,processed_transcript,1,13694
0610009L18Rik,11,120348678,120351190,bidirectional_promoter_lncRNA,1,2512
0610010F05Rik,11,23564961,23633639,protein_coding,-1,68678


In [9]:
_gene_df['chr']

'7'

In [20]:
# add a genomic_position column to facilitate downstream analysis
_chr_loc_list = []
for _gene_df in clean_gene_df.iloc():
    _chr_loc = 'chr' + _gene_df['chr'] + '_' + str(_gene_df['start']) + '_' + str(_gene_df['end'])
    _chr_loc_list.append(_chr_loc)
    
    
    
clean_gene_df_new = clean_gene_df.copy()
clean_gene_df_new['genomic_position'] = _chr_loc_list

clean_gene_df_new

,chr,start,end,gene_biotype,coding_strand,length,genomic_position
gene,,,,,,,
0610005C13Rik,7,45567794,45575327,antisense,-1,7533,chr7_45567794_45575327
0610009B22Rik,11,51685386,51688874,protein_coding,-1,3488,chr11_51685386_51688874
0610009E02Rik,2,26445696,26459390,processed_transcript,1,13694,chr2_26445696_26459390
0610009L18Rik,11,120348678,120351190,bidirectional_promoter_lncRNA,1,2512,chr11_120348678_120351190
0610010F05Rik,11,23564961,23633639,protein_coding,-1,68678,chr11_23564961_23633639
...,...,...,...,...,...,...,...
Zyg11b,4,108229724,108301096,protein_coding,-1,71372,chr4_108229724_108301096
Zyx,6,42349630,42360213,protein_coding,1,10583,chr6_42349630_42360213
Zzef1,11,72796226,72927120,protein_coding,1,130894,chr11_72796226_72927120


In [21]:
# save the queried Mop gene df

clean_gene_df_new.to_csv(os.path.join(pool_folder, "MOp_smart_sn_gene_chr_info_NEW_from_transcriptome_FORMAT.csv"), index=True)

The scripts above can be used to generate gene annotation as in:

[MOp_annotation_smart_seq](../../resources/MOp_smart_sn_gene_chr_info_NEW_from_transcriptome_FORMAT.csv)

[MOp_annotation_10x](../../resources/MOp_10x_snRNA_chr_info_NEW_from_transcriptome_FORMAT.csv)